In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install transformers
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# Enable text wrapping so we don't have to scrool horizontally
from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# A wrapper function whcih will get completion by the model from a user's query
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Question:
  {query}

  ### Answer:
  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)
  return (decoded[0])

In [15]:
# Import the necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Define custom quantization configuration for BitsAndBytes (BNB) quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                    # Load the model with 4-bit quantization
    bnb_4bit_use_double_quant=True,       # Use double quantization for 4-bit weights
    bnb_4bit_quant_type="nf4",           # Use nf4 quantization method
    bnb_4bit_compute_dtype=torch.bfloat16 # Compute with 4-bit quantized weights in bfloat16 data type
)

# Specify the pre-trained model identifier
model_id = "mistralai/Mistral-7B-v0.1"

# Load the pre-trained model with the specified quantization configuration
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

# Load the tokenizer for the same pre-trained model and add an end-of-sequence token
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
result = get_completion(query="Can you explain the basic principles behind financial modeling and forecasting?", model=model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s> 
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Question:
  Can you explain the basic principles behind financial modeling and forecasting?

  ### Answer:
  </s>щуипрвамо, financial modeling is defined like this:
  a mathematical tool which is used to get an estimate of a value over a period of time or a future value of an asset, for example, stock, future, property, or a company.
  So, let’s start with the first thing in financial modeling, which is “forecasting”.
  Forecasting simply means a prediction which is done using historical data. It can be a short-term or a longer-term prediction. It is used for several purposes, and the most important one is for determining the price of products (inventory), expenses of the firm, a change in demand for one’s products and a change in the rate of profit.
  Then there’s financial modeling that is just an analysis of the business to be done. In this, cash flows and income st

In [5]:
from datasets import load_dataset
data = load_dataset("gbharti/finance-alpaca", split='train[:10%]')


# Explore the data
df = data.to_pandas()
df.head(10)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

,input,output,instruction,text
0,,The car deal makes money 3 ways. If you pay in...,"For a car, what scams can be plotted with 0% f...",
1,,"That is kind of the point, one of the hopes is...",Why does it matter if a Central Bank has a neg...,
2,,"Pay off your debt. As you witnessed, no ""inve...",Where should I be investing my money?,
3,,"Equity options, at least those traded in the A...",Specifically when do options expire?,
4,,"Automatic exercisions can be extremely risky, ...",Negative Balance from Automatic Options Exerci...,
5,,"Generally ""default"" means that the company can...",Approximation of equity value for company in d...,
6,,The game is not zero sum. When a friend and I ...,Is it true that 90% of investors lose their mo...,
7,,"In general, you can only be charged for servic...",Can a company charge you for services never re...,
8,,Being self employed just means you fill out so...,Working out if I should be registered as self-...,
9,,"For eToro, just like any other brokerage firm,...",About eToro investments,


In [6]:
# Define a function to generate a prompt text based on a data point
def generate_prompt(data_point):
    """
    Generate input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenized prompt
    """
    # Check if the data point has additional context information
    if data_point['input']:
        # Create a text with instruction, input, and response
        text = 'Below is an instruction that describes a task, paired with an input that provides' \
               ' further context. Write a response that appropriately completes the request.\n\n'
        text += f'### Instruction:\n{data_point["instruction"]}\n\n'
        text += f'### Input:\n{data_point["input"]}\n\n'
        text += f'### Response:\n{data_point["output"]}'

    # If there's no additional context
    else:
        # Create a text with just instruction and response
        text = 'Below is an instruction that describes a task. Write a response that ' \
               'appropriately completes the request.\n\n'
        text += f'### Instruction:\n{data_point["instruction"]}\n\n'
        text += f'### Response:\n{data_point["output"]}'
    return text

# Add the "prompt" column in the dataset by applying the generate_prompt function to each data point
text_column = [generate_prompt(data_point) for data_point in data]
data = data.add_column("prompt", text_column)

# Shuffle the dataset with a specified seed
data = data.shuffle(seed=1234)

# Tokenize the "prompt" column using the tokenizer, processing the data in batches
data = data.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

# Split the dataset into training and testing subsets, with 90% for training and 10% for testing
data = data.train_test_split(test_size=0.1)
train_data = data["train"]
test_data = data["test"]

Map:   0%|          | 0/6891 [00:00<?, ? examples/s]

In [14]:
# Import the necessary function from the "peft" library to prepare a model for k-bit training
from peft import prepare_model_for_kbit_training

# Enable gradient checkpointing for the model
model.gradient_checkpointing_enable()

# Prepare the model for k-bit training using the "prepare_model_for_kbit_training" function
model = prepare_model_for_kbit_training(model)

# Define a function to print the number of trainable parameters in the model
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    # Iterate through model parameters
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    # Print the number of trainable parameters, total parameters, and the percentage of trainable parameters
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

# Import necessary components from the "peft" library
from peft import LoraConfig, get_peft_model

# Define a configuration for the LoRA (Learnable Requantization Activation) method
lora_config = LoraConfig(
    r=8,                                   # Number of quantization levels
    lora_alpha=32,                         # Hyperparameter for LoRA
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], # Modules to apply LoRA to
    lora_dropout=0.05,                     # Dropout probability
    bias="none",                           # Type of bias
    task_type="CAUSAL_LM"                  # Task type (in this case, Causal Language Modeling)
)

# Get a model with LoRA applied to it using the defined configuration
peft_model = get_peft_model(model, lora_config)

# Print the number of trainable parameters in the model after applying LoRA
print_trainable_parameters(peft_model)

trainable params: 6815744 || all params: 3758886912 || trainable%: 0.18132346515244138


In [12]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
# Import the necessary libraries from the transformers package
import transformers

# Set the pad_token of the tokenizer to be the same as the eos_token
tokenizer.pad_token = tokenizer.eos_token

# Create a trainer for fine-tuning a model
trainer = transformers.Trainer(
    model=model,  # The model to be trained
    train_dataset=train_data,  # Training dataset
    eval_dataset=test_data,    # Evaluation dataset
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,        # Batch size per device during training
        gradient_accumulation_steps=4,        # Number of gradient accumulation steps
        warmup_steps=0.03,                    # Number of warm-up steps for learning rate
        max_steps=100,                        # Maximum number of training steps
        learning_rate=2e-4,                   # Learning rate
        fp16=True,                            # Enable mixed-precision training
        logging_steps=1,                      # Logging frequency during training
        output_dir="outputs_mistral_b_finance_finetuned_test",  # Directory to save output files
        optim="paged_adamw_8bit",             # Optimizer type
        save_strategy="epoch",                # Strategy for saving checkpoints
        push_to_hub=True                      # Push to the Hugging Face model hub
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    # Data collator for language modeling task
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
model.push_to_hub("mistral_b_finance_finetuned_test")
tokenizer.push_to_hub("mistral_b_finance_finetuned_test")

In [19]:
# Import necessary libraries
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# Specifying thr path of my fien tuned model in my huggingface directory
peft_model_id = "ayman56/mistral_b_finance_finetuned_test"

# Load the configuration for the Peft model
config = PeftConfig.from_pretrained(peft_model_id)

# Load the base model for Causal Language Modeling with specified settings
# - Use AutoModelForCausalLM to load the model.
# - Set return_dict=True to enable returning model outputs as dictionaries.
# - Enable 4-bit weight quantization with load_in_4bit=True.
# - Set device_map='auto' to automatically allocate the model on available devices.
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    load_in_4bit=True,
    device_map='auto'
)

# Load the tokenizer associated with the base model
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the PeftModel, which applies the Peft (Perturbed Embeddings for Few-shot Text Classification) method
# to the base model, using the specified pre-trained Peft model identifier
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

In [20]:
result = get_completion(query="Will capital gains affect my tax bracket?", model=model, tokenizer=tokenizer)
print(result)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


<s> 
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Question:
  Can you explain the basic principles behind financial modeling and forecasting?

  ### Answer:
   Financial forecasting  isn't always about predictions -  In fact I tend to see a difference with the term "financial modelling".  Generally I view a forecaster's job as providing estimates of the future state of a project or enterprise or individual. In fact the term "forecast" is not a prediction of the future at all, it is more of an assessment of where we are and how things are tracking.  There are a number of things that make such forecasts possible. Obviously some things like population data are available as a baseline that we can extrapolate from; these tend to be things we already have data on.  It's also a lot easier to forecast things involving natural phenomenon. The only people doing weather forecasts are the meteorologists - nobody thinks a geologis

In [21]:
result = get_completion(query="Why do we pay taxes ?", model=model, tokenizer=tokenizer)
print(result)

<s> 
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Question:
  Why do we pay taxes ?

  ### Answer:
   If we didn’t the government would not have the money to pay the people it hired  to provide the various services the government is charged with providing. Governments are not profit-driven, and generally do not produce anything that provides a profit. The government will generally provide things (roads, highways, bridges, public schools/universities, sewage lines, etc) that increase your property value, in exchange for an ever-increasing property tax,  and may also provide things (fire departments, police departments, military, etc) that reduce the amount of damage you may suffer at the hands of people that you otherwise could not defend yourself against, in exchange for an ever-increasing income tax.  It’s your choice: pay taxes and get some degree of public services, or decline to pay taxes and pay for those service